# Loading Processed Data

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for pandas
pd.set_option('display.max_columns', None)

# Load the dataset
filepath = 'data/processed/insurance_claims_processed_mode_two.csv'

df = pd.read_csv(filepath)
df_copy = df.copy()

# Display the first few rows of the copied DataFrame
df_copy.head()

,policy_deductable,policy_annual_premium,umbrella_limit,capital-gains,capital-loss,incident_severity,incident_hour_of_the_day,total_claim_amount,auto_year,fraud_reported,day,insured_occupation_armed-forces,insured_occupation_craft-repair,insured_occupation_exec-managerial,insured_occupation_farming-fishing,insured_occupation_handlers-cleaners,insured_occupation_machine-op-inspct,insured_occupation_other-service,insured_occupation_priv-house-serv,insured_occupation_prof-specialty,insured_occupation_protective-serv,insured_occupation_sales,insured_occupation_tech-support,insured_occupation_transport-moving,insured_hobbies_basketball,insured_hobbies_board-games,insured_hobbies_bungie-jumping,insured_hobbies_camping,insured_hobbies_chess,insured_hobbies_cross-fit,insured_hobbies_dancing,insured_hobbies_exercise,insured_hobbies_golf,insured_hobbies_hiking,insured_hobbies_kayaking,insured_hobbies_movies,insured_hobbies_paintball,insured_hobbies_polo,insured_hobbies_reading,insured_hobbies_skydiving,insured_hobbies_sleeping,insured_hobbies_video-games,insured_hobbies_yachting,auto_model_92x,auto_model_93,auto_model_95,auto_model_A3,auto_model_A5,auto_model_Accord,auto_model_C300,auto_model_CRV,auto_model_Camry,auto_model_Civic,auto_model_Corolla,auto_model_E400,auto_model_Escape,auto_model_F150,auto_model_Forrestor,auto_model_Fusion,auto_model_Grand Cherokee,auto_model_Highlander,auto_model_Impreza,auto_model_Jetta,auto_model_Legacy,auto_model_M5,auto_model_MDX,auto_model_ML350,auto_model_Malibu,auto_model_Maxima,auto_model_Neon,auto_model_Passat,auto_model_Pathfinder,auto_model_RAM,auto_model_RSX,auto_model_Silverado,auto_model_TL,auto_model_Tahoe,auto_model_Ultima,auto_model_Wrangler,auto_model_X5,auto_model_X6
0,0.333333,0.590075,0.0,0.530348,1.000000,0.5,0.217391,0.553137,0.45,1,0.800000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.594061,0.5,0.349254,1.000000,0.0,0.304348,0.171791,0.60,0,0.700000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.595725,0.6,0.486567,0.438344,0.5,0.217391,0.468428,0.95,1,0.300000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.333333,0.554366,0.0,0.000000,1.000000,0.5,0.826087,0.475650,0.40,1,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.333333,0.543009,0.0,0.000000,0.306931,0.0,0.000000,0.625774,0.85,0,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Model Development

### Handling Class Imbalance

In [2]:
# Data Split into X and y
X = df_copy.drop(columns = ['fraud_reported'])
y = df_copy['fraud_reported']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state= 69)

# Imbalance Handling using SMOTETomek
from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state= 69)

X_train_smt, y_train_smt = smt.fit_resample(X_train,y_train)

### Logistic Regression

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model_log = LogisticRegression(
    penalty= 'l2', # Ridge regularization
    C= 1.0, # Regularization strength
    solver= 'lbfgs', # Algorithm to use for optimization
)
model_log.fit(X_train_smt,  y_train_smt)
y_pred = model_log.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.65      0.72       182
           1       0.35      0.54      0.43        65

    accuracy                           0.62       247
   macro avg       0.58      0.59      0.57       247
weighted avg       0.68      0.62      0.64       247



### Support Vector Machine

In [5]:
from sklearn.svm import SVC

model_svc = SVC(
    kernel = 'rbf',
    C= 1.0,
    gamma= 'scale'
)
model_svc.fit(X_train_smt,  y_train_smt)
y_pred = model_svc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       182
           1       0.55      0.43      0.48        65

    accuracy                           0.76       247
   macro avg       0.68      0.65      0.66       247
weighted avg       0.74      0.76      0.75       247



### Decision Tree

In [6]:
from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier(
    criterion= 'gini',
    splitter= 'best',
    max_depth = None,
    min_samples_leaf = 1,
    max_features= None, 
    random_state= 69
)
model_dt.fit(X_train_smt,  y_train_smt)
y_pred = model_dt.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.83      0.85       182
           1       0.57      0.63      0.60        65

    accuracy                           0.78       247
   macro avg       0.72      0.73      0.72       247
weighted avg       0.79      0.78      0.78       247



### Majority Voting

In [7]:
from sklearn.ensemble import VotingClassifier

estimators = [
    ('Logistic', model_log),
    ('SVC', SVC(probability= True)),
    ('DTC', model_dt),
]

model_mv = VotingClassifier(
    estimators = estimators,
    voting= 'soft'
)
model_mv.fit(X_train_smt,  y_train_smt)
y_pred = model_mv.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.87      0.86       182
           1       0.60      0.54      0.57        65

    accuracy                           0.79       247
   macro avg       0.72      0.71      0.71       247
weighted avg       0.78      0.79      0.78       247



### Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(
    n_estimators = 100,
    criterion= 'gini',
    max_depth = None,
    min_samples_leaf = 1,
    max_features= None, 
    random_state= 69
)
model_rf.fit(X_train_smt,  y_train_smt)
y_pred = model_rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.81      0.85       182
           1       0.59      0.77      0.67        65

    accuracy                           0.80       247
   macro avg       0.75      0.79      0.76       247
weighted avg       0.82      0.80      0.81       247



### Extreme Gradient Boost

In [9]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier()
model_xgb.fit(X_train_smt,  y_train_smt)
y_pred = model_xgb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.82      0.84       182
           1       0.57      0.66      0.61        65

    accuracy                           0.78       247
   macro avg       0.72      0.74      0.73       247
weighted avg       0.79      0.78      0.78       247



# Para. Fine Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],       # Number of trees
    'max_depth': [None, 10, 20, 30],       # Depth of trees
    'min_samples_split': [2, 5, 10],       # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4],         # Minimum samples per leaf
    'max_features': ['sqrt', 'log2'],      # Number of features to consider
    'bootstrap': [True, False]             # Whether bootstrap samples are used
}

grid_search = GridSearchCV(
    estimator=model_rf,
    param_grid=param_grid,
    cv=5,                       # 5-fold cross-validation
    scoring='recall',           # Can change to 'f1', 'roc_auc', etc.
    n_jobs=-1,                  # Use all available cores
    verbose=2
)

grid_search.fit(X_train_smt,  y_train_smt)

best_model = grid_search.best_estimator_
best_model

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


RandomForestClassifier(max_depth=20, n_estimators=200, random_state=69)

#### Evaluating Performance

In [43]:
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.87      0.86       182
           1       0.60      0.54      0.57        65

    accuracy                           0.79       247
   macro avg       0.72      0.71      0.71       247
weighted avg       0.78      0.79      0.78       247



- Since, fine tuning didn't improved performance we will use default configurations.

# Export Model

In [ ]:
import joblib

joblib.dump(
    model_rf,
    'model/model_obj.pkl'
)

['D:/[01] SKILLS/[00] Projects/08 Fraud Claims Prediction/model/model_obj.pkl']